In [2]:
%run tracker.py.ipynb
import numpy as np
import cv2
import pickle
import glob
#from tracker import tracker

In [3]:
# From tracker import tracker

In [4]:
dist_pickle = pickle.load(open('calibration_pickle.p','rb'))

In [5]:
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']

In [6]:
# play list number 14:27
# https://www.youtube.com/watch?v=vWY8YUayf9Q&feature=youtu.be
    

In [7]:
def abs_sobel_thresh(img,orient='x',sobel_kernel=3,thresh=(0,255)):
    # calculate directional gradient
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    if orient =='x':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV_64F,1,0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray,cv2.CV_64F,0,1))
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    
    binary_output[(scaled_sobel>=thresh[0])&(scaled_sobel<=thresh[1])] = 1
    return binary_output


In [8]:
def mag_thresh(image,sobel_kernel = 3,mag_thresh =(0,255)):
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    gradmag = np.sqrt(sobelx**2,sobely**2)
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.unit8)
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

In [9]:
def dir_threshold(image, sobel_kernel = 3, thresh = (0,np.pi/2)):
    # calculate gradient direction
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    with np.errstate(divide='ignore',invaid='ignore'):
        absgraddir = np.absolute(np.arctan(sobely/sobelx))
        binary_output = np.zeros_like(absgraddir)
        #apply threshold
        binary_output[(absgraddir >= thresh[0])&(absgraddir <= thresh[1])]=1
    return binary_output
    
    

In [10]:
def color_threshold(image,sthresh = (0,255),vthresh = (0,255)):
    hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0])&(s_channel <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >=vthresh[0])&(v_channel <= vthresh[1])] = 1
    
    output = np.zeros_like(s_channel)
    output[(s_binary ==1)&(v_binary ==1)] = 1
    return output



In [11]:
def window_mask(width,height,img_ref,center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width)):min(int(center+width),img_ref.shape[1])] = 1
    return output

In [12]:
# make a list of test images
images = glob.glob('test_images/test*.jpg')


In [13]:
for idx,fname in enumerate(images):
    # read in image
    img = cv2.imread(fname)
    # undistort the image
    img = cv2.undistort(img,mtx,dist,None,mtx)
    
    
#     preprocessImage = np.zeros_like(img[:,:,0])
#     gradx = abs_sobel_thresh(img, orient = 'x', thresh =(12,255)) 
#     grady = abs_sobel_thresh(img, orient = 'y', thresh =(25,255))
#     c_binary = color_threshold(img, sthresh = (100,255), vthresh = (50,255))
#     preprocessImage[((gradx == 1)&(grady == 1)|(c_binary == 1))] = 255
    
#     img_size =(img.shape[1],img.shape[0])
#     bot_width = .76
#     mid_width = .08
#     height_pct= .62
#     bottom_trim= .935
    
 
#     offset = img_size[0]*.25
#     src = np.float32([[img.shape[1]*(.5-mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(.5+mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(.5+bot_width/2),img.shape[0]*bottom_trim],[img.shape[1]*(.5-bot_width/2),img.shape[0]*bottom_trim]])
#     dst = np.float32([[offset, 0], [img_size[0]-offset, 0],[img_size[0]-offset, img_size[1]],[offset, img_size[1]]])
    
#     M = cv2.getPerspectiveTransform(src, dst)
#     Minv = cv2.getPerspectiveTransform(dst, src)                 
#     warped = cv2.warpPerspective(preprocessImage, M, img_size, flags=cv2.INTER_LINEAR)
    
#     window_width = 25
#     window_height = 80
    
#     #set up the overall class to do all the tracking 
#     curve_centers = tracker(Mywindow_width = window_width, Mywindow_height = window_height, Mymargin = 25, My_ym = 10/720, My_xm = 4/384,Mysmooth_factor = 15)
    
#     window_centroids = curve_centers.find_window_centroids(warped)
    
    
#     # points used to draw all the left and right windows
    
#     l_points = np.zeros_like(warped)
#     r_points = np.zeros_like(warped)
    
#     # points used to find the left and right lanes
    
#     rightx = []
#     leftx = []
    
    
#     # Go through each level and draw the windows
    
#     for level in range(0,len(window_centroids)):
        
#         leftx.append(window_centroids[level][0])
#         rightx.append(window_centroids[level][1])
        
#         # window_mask is a function to draw window areas
#         l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0], level)
#         r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1], level)
        
#         # add center value found in frame to the list of lane points per left,right
        

        
#         # add graphic points from window mask here to total pixels found
        
#         l_points[(l_points == 255)|((l_mask == 1))] = 255
#         r_points[(r_points == 255)|((r_mask == 1))] = 255
        
    
#     template = np.array(r_points+l_points,np.uint8)
#     zero_channel = np.zeros_like(template)
#     template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8)
#     warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8)
#     result = cv2.addWeighted(warpage,1,template,0.5,0.0)
                            
#     # result = warped
    
    
#     yvals = range(0,warped.shape[0])
    
#     res_yvals = np.arange(warped.shape[0]-(window_height/2),0,-window_height)
    
#     left_fit = np.polyfit(res_yvals,leftx,2)
#     left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
#     left_fitx = np.array(left_fitx,np.int32)
    
    
#     right_fit = np.polyfit(res_yvals,rightx,2)
#     right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
#     right_fitx = np.array(right_fitx,np.int32)
    
#     left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2),axis= 0),np.concatenate((yvals,yvals[::-1]),axis = 0))),np.int32)
#     right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis= 0),np.concatenate((yvals,yvals[::-1]),axis = 0))),np.int32)
#     middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis = 0),np.concatenate((yvals,yvals[::-1]),axis = 0))),np.int32)
    
#     road = np.zeros_like(img)
#     road_bkg = np.zeros_like(img)
#     cv2.fillPoly(road,[left_lane],color = [255,0,0])
#     cv2.fillPoly(road,[right_lane],color = [0,0,255])
#     cv2.fillPoly(road_bkg,[left_lane],color=[255,255,255])
#     cv2.fillPoly(road_bkg,[right_lane],color = [255,255,255])
    
#     road_warped = cv2.warpPerspective(road,Minv,img_size,flags = cv2.INTER_LINEAR)
#     road_warped_bkg = cv2.warpPerspective(road_bkg,Minv,img_size,flags = cv2.INTER_LINEAR)
    
#     base = cv2.addWeighted(img,1.0,road_warped_bkg,-1.0,0.0)
#     result = cv2.addWeighted(base,1.0,road_warped,1,0.0)
    
    result = img

                        
#     write_name = 'test_images/tracked'+str(idx)+'.jpg'
    write_name = 'test_images/undistort'+str(idx)+'.jpg'
    
    cv2.imwrite(write_name,result)